In [0]:
dbutils.fs.rm('dbfs:/FileStore/train/pothole', True)

In [0]:
pip install tensorflow==2.11.*

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [0]:
dbutils.fs.mount(
  source = "wasbs://pothole-poc-dataset@zee.blob.core.windows.net",
  mount_point = "/mnt/iotdata",
  extra_configs = {"fs.azure.account.key.zee.blob.core.windows.net":"YlBjJI7gIJpTUQ5ELXbqiqI/T/WhLRDi03zTBEiVU30O0O9zXwkWqKULPst5k+5cors7NYTbyw/v+AStJ3bIBw=="})

In [0]:
dbutils.fs.ls("dbfs:/mnt/iotdata/train")

In [0]:
# pothole and normal data path pre processing

normal_train_path = '/dbfs/mnt/iotdata/train/normal'

pothole_train_path = '/dbfs/mnt/iotdata/train/pothole'

normal_test_path = '/dbfs/mnt/iotdata/test/normal'

pothole_test_path = '/dbfs/mnt/iotdata/test/pothole'

In [0]:
# data_preP dir


normal_train_path_final =  '/dbfs/mnt/iotdata/right-formate-data/train/zeent'

pothole_train_path_final = '/dbfs/mnt/iotdata/right-formate-data/train/zeept'

normal_test_path_final = '/dbfs/mnt/iotdata/right-formate-data/test/zeenv'

pothole_test_path_final = '/dbfs/mnt/iotdata/right-formate-data/test/zeepv'

In [0]:
pip install Pillow

In [0]:
import os

In [0]:
from PIL import Image

for i in os.listdir(normal_train_path):
  dir = normal_train_path + '/' + i
  im = Image.open(dir)
  rgb_im = im.convert('RGB')
  rgb_im.save(normal_train_path_final + '/' + i)

In [0]:
from PIL import Image

for i in os.listdir(pothole_train_path):
  dir = pothole_train_path + '/' + i
  im = Image.open(dir)
  rgb_im = im.convert('RGB')
  rgb_im.save(pothole_train_path_final + '/' + i)

In [0]:
from PIL import Image

for i in os.listdir(normal_test_path):
  dir = normal_test_path + '/' + i
  im = Image.open(dir)
  rgb_im = im.convert('RGB')
  rgb_im.save(normal_test_path_final + '/' + i)

In [0]:
from PIL import Image

for i in os.listdir(pothole_test_path):
  dir = pothole_test_path + '/' + i
  im = Image.open(dir)
  rgb_im = im.convert('RGB')
  rgb_im.save(pothole_test_path_final + '/' + i)

In [0]:
# from PIL import Image
# import os

# list_path =[normal_train_path, pothole_train_path, normal_test_path, pothole_test_path]
# final_path = [normal_train_path_final, pothole_train_path_final, normal_test_path_final, pothole_test_path_final]

# for j in list_path:
#   for k in final_path:
#     for i in os.listdir(j):
#       dir = j + '/' + i
#       im = Image.open(dir)
#       rgb_im = im.convert('RGB')
#       rgb_im.save(k + '/' + i )


In [0]:
list_path =[normal_train_path, pothole_train_path, normal_test_path, pothole_test_path]
final_path = [normal_train_path_final, pothole_train_path_final, normal_test_path_final, pothole_test_path_final]

In [0]:
from PIL import Image

for i,j in enumerate(list_path):
    for k in os.listdir(j):
      dir = j + '/' + k
      im = Image.open(dir)
      rgb_im = im.convert('RGB')
      rgb_im.save(final_path[i] + '/' + k )

In [0]:
import shutil

In [0]:
shutil.rmtree('/dbfs/mnt/iotdata/right-formate-data/train/normal')

In [0]:
sa = dbutils.fs.ls(normal_train_path)

In [0]:
from PIL import Image
import os

In [0]:
 for i in sa:
        im = Image.open(i)
        C = im.convert('RGB')
        print(c)
        break

In [0]:
Image.open('dbfs:/mnt/iotdata/train/normal/100.jpg')

In [0]:
# generators
from tensorflow import keras

train_ds = keras.utils.image_dataset_from_directory(
    directory = '/dbfs/mnt/iotdata/right-formate-data/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/dbfs/mnt/iotdata/right-formate-data/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

In [0]:
# Normalize
import tensorflow as tf
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [0]:
# create CNN model
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout


model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [0]:
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout


model.summary()

In [0]:
model.summary()

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history = model.fit(train_ds,epochs=10,validation_data=validation_ds)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [0]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [0]:
pip install opencv-python

In [0]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [0]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(256,256,3)
)

In [0]:
conv_base.summary()

In [0]:
model_1 = Sequential()

model_1.add(conv_base)
model_1.add(Flatten())
model_1.add(Dense(256,activation='relu'))
model_1.add(Dense(1,activation='sigmoid'))

In [0]:
model_1.summary()

In [0]:
conv_base.trainable = False

In [0]:
model_1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
history_1 = model_1.fit(train_ds,epochs=10,validation_data=validation_ds)

In [0]:
import matplotlib.pyplot as plt

plt.plot(history_1.history['accuracy'],color='red',label='train')
plt.plot(history_1.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [0]:
plt.plot(history_1.history['loss'],color='red',label='train')
plt.plot(history_1.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [0]:
import cv2

In [0]:
test_img = cv2.imread('/dbfs/mnt/iotdata/right-formate-data/test/normal/39.jpg')

In [0]:
plt.imshow(test_img)

In [0]:
test_img.shape

In [0]:
test_img = cv2.resize(test_img,(256,256))

In [0]:
test_input = test_img.reshape((1,256,256,3))

In [0]:
model_1.predict(test_input)

In [0]:
s= [2,3,4,5,6,7]
v= [4,6,7,8,9,9]

u = [9,8,7,6,5,4]

In [0]:
for i,j in enumerate(s):
    for k in u:
        print(j*v[i]*k)

In [0]:
for i in s:
    for j in v:
        for k in u:
            print(i*j*k)
        break
